In [ ]:
import numpy as np
import pandas as pd
import glob
import os.path
import datetime
import os
import geopandas as gpd
from geopy.distance import great_circle
from datetime import datetime, timedelta
import shapely as shp

!pip install movingpandas
import movingpandas as mpd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# The dataset

(_copied_ from [url](https://www.microsoft.com/en-us/download/details.aspx?id=52367))

> This GPS trajectory dataset was collected in (Microsoft Research Asia) Geolife project by 182 users in a period of over three years (from April 2007 to August 2012). A GPS trajectory of this dataset is represented by a sequence of time-stamped points, each of which contains the information of latitude, longitude and altitude. This dataset contains 17,621 trajectories with a total distance of about 1.2 million kilometers and a total duration of 48,000+ hours. These trajectories were recorded by different GPS loggers and GPS-phones, and have a variety of sampling rates. 91 percent of the trajectories are logged in a dense representation, e.g. every 1\~5 seconds or every 5\~10 meters per point. This dataset recoded a broad range of users’ outdoor movements, including not only life routines like go home and go to work but also some entertainments and sports activities, such as shopping, sightseeing, dining, hiking, and cycling.

# Formating `.plt` files into trajectory instances

The dataset can be downloaded from [Microsoft](https://www.microsoft.com/en-us/research/publication/geolife-gps-trajectory-dataset-user-guide/). Its structure is as follows:

```
- Data Trajectories 1.3/
    - Data/
        - 000/
           - Trajectory/
               - <timestamp1>.plt
               - <timestamp2>.plt
        - 001/
        - ...
        - 180/
        - 181/
    - User Guide-1.3.pdf
```

000, 001, ..., 180, 181 are the users that recorded their trajectories, i.e., 182 users. Many of them did not label the activity, e.g., walking, bus, car, plane, taxi, and thus are later excluded.

Each file inside `Trajectory` is a single user trajectory, containing many timestamped GPS recordings, which will receive its unique identifier later on.

The following script from [heremaps.github.io](https://heremaps.github.io/pptk/tutorials/viewer/geolife.html) aggregates  all the `.plt` data and converts them into a pandas dataframe.

In [ ]:
import numpy as np
import pandas as pd
import glob
import os.path
import datetime
import os

def read_plt(plt_file):
    points = pd.read_csv(plt_file, skiprows=6, header=None,
                         parse_dates=[[5, 6]], infer_datetime_format=True)

    # for clarity rename columns
    points.rename(inplace=True, columns={'5_6': 'time', 0: 'lat', 1: 'lon', 3: 'alt'})

    # remove unused columns
    points.drop(inplace=True, columns=[2, 4])

    return points

mode_names = ['walk', 'bike', 'bus', 'car', 'subway','train', 'airplane', 'boat', 'run', 'motorcycle', 'taxi']
mode_ids = {s : i + 1 for i, s in enumerate(mode_names)}

def read_labels(labels_file):
    labels = pd.read_csv(labels_file, skiprows=1, header=None,
                         parse_dates=[[0, 1], [2, 3]],
                         infer_datetime_format=True, delim_whitespace=True)

    # for clarity rename columns
    labels.columns = ['start_time', 'end_time', 'label']

    # replace 'label' column with integer encoding
    labels['label'] = [mode_ids[i] for i in labels['label']]

    return labels

def apply_labels(points, labels):
    indices = labels['start_time'].searchsorted(points['time'], side='right') - 1
    no_label = (indices < 0) | (points['time'].values >= labels['end_time'].iloc[indices].values)
    points['label'] = labels['label'].iloc[indices].values
    points['label'][no_label] = 0

def read_user(user_folder):
    labels = None
    user_id = int(os.path.basename(user_folder))

    plt_files = glob.glob(os.path.join(user_folder, 'Trajectory', '*.plt'))
    dfs = []

    for traj_id, plt_file in enumerate(plt_files):
        df = read_plt(plt_file)
        df['trajectory_id'] = f"{user_id}_{traj_id}"  # unique trajectory ID
        dfs.append(df)

    df = pd.concat(dfs, ignore_index=True)

    labels_file = os.path.join(user_folder, 'labels.txt')
    if os.path.exists(labels_file):
        labels = read_labels(labels_file)
        apply_labels(df, labels)
    else:
        df['label'] = 0

    df['user'] = user_id
    return df


def read_all_users(folder):
    subfolders = os.listdir(folder)
    dfs = []
    for i, sf in enumerate(subfolders):
        print('[%d/%d] processing user %s' % (i + 1, len(subfolders), sf))
        df = read_user(os.path.join(folder,sf))
        df['user'] = int(sf)
        dfs.append(df)
    return pd.concat(dfs)

df = read_all_users("Geolife_Trajectories/Data")
df.to_csv("/content/drive/MyDrive/geolife.csv", index=False)


In [3]:
df = df[df['label'] > 0] # removing trajetories without any label
df = df[df['lat'] <= 90] # cleaning up impossible latitudes
df['time'] = pd.to_datetime(df['time'])

In [12]:
df.size

37989812

# Agregating trajectories and enriching the dataset

Right now, each df row is a timestamped GPS record, and can be aggregated into a single trajectory by the field `trajectory_id`. By doing this, we can derive features such as `speed`, `duration`, `distance`, etc.

All the following features below are incorporated into the new aggregated dataset:

| Field                | Description                                                                 | Data Type   | Feature Type         |
|----------------------|-----------------------------------------------------------------------------|-------------|----------------------|
| `user`               | Unique identifier of the user who performed the trajectory.                 | `int` / `str`| Metadata |
| `trajectory_id`      | Unique identifier for the trajectory within each user.                      | `int`        | Metadata |
| `start_time`          | Timestamp of when the trajectory started.                                  | `datetime`  | Temporal             |
| `end_time`          | Timestamp of when the trajectory ended.                                  | `datetime`  | Temporal             |
| `duration_s`          | Total duration of the trajectory in seconds.                               | `float`     | Temporal             |
| `start_hour`          | Hour of the day when the trajectory began (0–23).                          | `int`       | Temporal             |
| `weekday`             | Day of the week when the trajectory began (0 = Monday, 6 = Sunday).        | `int`       | Temporal             |
| `distance_m`          | Total distance traveled in meters.                                         | `float`     | Spatial              |
| `straightness_ratio`  | Ratio of straight-line to actual distance (1 = perfectly straight).        | `float`     | Spatial / Efficiency |
| `avg_speed_mps`       | Average speed in meters per second.                                        | `float`     | Movement             |
| `speed_std`           | Standard deviation of speeds during the trajectory.                        | `float`     | Variability          |
| `speed_entropy`       | Entropy of the speed distribution.                                         | `float`     | Variability          |
| `speed_peak_ratio`    | Ratio of average to maximum speed.                                         | `float`     | Movement / Load      |
| `movement_ratio`      | Proportion of time spent moving vs total time.                             | `float`     | Behavioral           |
| `load_factor`         | Ratio of average hourly activity to the peak hour.                         | `float`     | Behavioral           |
| `time_entropy`        | Entropy of hourly activity distribution.                                   | `float`     | Temporal Behavior    |
| `vcr`                 | Velocity Change Rate (speed jumps per km).                                 | `float`     | Movement Dynamics    |
| `sr`                  | Stop Rate (number of stops per km).                                        | `float`     | Stop Behavior        |
| `hcr`                 | Heading Change Rate (sharp direction changes per km).                      | `float`     | Route Behavior       |
| `num_points`          | Number of GPS points in the trajectory.                                    | `int`       | Metadata             |
| `coordinates`         | Line geometry of the trajectory.                                           | `LineString`| Spatial Geometry     |
| `centroid`            | Geometric center of the trajectory path.                                   | `Point`     | Spatial Geometry     |
| `label`               | Most frequent mode of transport in the trajectory.                         | `int` / `str`| Transport Mode      |
| `labels`              | List of all transport mode labels observed in the trajectory.              | `list`      | Transport Mode       |



In [13]:
import pandas as pd
import numpy as np
from shapely.geometry import LineString
import math
from scipy.stats import entropy
from geopy.distance import great_circle

def compute_entropy(values, bins=10):
    if len(values) < 2:
        return 0
    hist, _ = np.histogram(values, bins=bins)
    probs = hist / np.sum(hist)
    return entropy(probs, base=2)

def compute_heading(lat1, lon1, lat2, lon2):
    dLon = math.radians(lon2 - lon1)
    lat1 = math.radians(lat1)
    lat2 = math.radians(lat2)
    x = math.sin(dLon) * math.cos(lat2)
    y = math.cos(lat1) * math.sin(lat2) - math.sin(lat1) * math.cos(lat2) * math.cos(dLon)
    bearing = math.atan2(x, y)
    bearing = math.degrees(bearing)
    return (bearing + 360) % 360

def total_distance(latitudes, longitudes):
    dist = 0.0
    for i in range(1, len(latitudes)):
        coord1 = (latitudes[i-1], longitudes[i-1])
        coord2 = (latitudes[i], longitudes[i])
        dist += great_circle(coord1, coord2).meters
    return dist

def aggregate_activity(group, vcr_thresh=1.0, stop_thresh=0.5, hcr_thresh=30):
    if len(group) < 2:
        return None

    group = group.sort_values('time')
    latitudes = group['lat'].values
    longitudes = group['lon'].values
    times = pd.to_datetime(group['time']).values.astype('datetime64[s]').astype(np.int64)
    times_seconds = times

    # compute total distance and line geometry
    distance = total_distance(latitudes, longitudes)
    line = LineString(list(zip(latitudes, longitudes)))
    displacement = great_circle((latitudes[0], longitudes[0]), (latitudes[-1], longitudes[-1])).meters
    straightness = displacement / distance if distance > 0 else 0

    # compute speeds between points
    speeds = []
    time_diffs = []
    for i in range(1, len(times_seconds)):
        dt = times_seconds[i] - times_seconds[i-1]
        if dt > 0:
            d = great_circle((latitudes[i-1], longitudes[i-1]), (latitudes[i], longitudes[i])).meters
            speeds.append(d / dt)
            time_diffs.append(dt)
        else:
            speeds.append(0)
            time_diffs.append(0)

    speeds = np.array(speeds)
    time_diffs = np.array(time_diffs)
    duration = np.sum(time_diffs)

    # VCR, SR, HCR
    speed_changes = 0
    stops = 0
    heading_changes = 0
    prev_heading = None
    prev_speed = None

    for i in range(len(speeds)):
        speed = speeds[i]
        if prev_speed is not None and abs(speed - prev_speed) > vcr_thresh:
            speed_changes += 1
        if speed < stop_thresh:
            stops += 1

        if i < len(latitudes) - 1:
            heading = compute_heading(latitudes[i], longitudes[i], latitudes[i+1], longitudes[i+1])
            if prev_heading is not None:
                delta_heading = abs(heading - prev_heading)
                if delta_heading > hcr_thresh:
                    heading_changes += 1
            prev_heading = heading

        prev_speed = speed

    norm_dist = distance / 1000 if distance > 0 else 1

    speed_std = np.std(speeds) if speeds.size > 0 else 0
    speed_entropy = compute_entropy(speeds, bins=10)
    avg_speed = np.mean(speeds) if speeds.size > 0 else 0
    max_speed = np.max(speeds) if speeds.size > 0 else 0
    speed_peak_ratio = avg_speed / max_speed if max_speed > 0 else 0

    moving_time = np.sum(time_diffs[speeds >= stop_thresh]) if len(time_diffs) > 0 else 0
    movement_ratio = moving_time / duration if duration > 0 else 0

    # hourly distribution for load factor and time entropy
    group['hour'] = group['time'].dt.hour
    hourly_counts = group.groupby('hour').size()
    if len(hourly_counts) > 0 and hourly_counts.max() > 0:
        load_factor = hourly_counts.mean() / hourly_counts.max()
    else:
        load_factor = 0
    time_entropy = compute_entropy(group['hour'], bins=24)

    start_time = group['time'].min()
    start_hour = start_time.hour
    weekday = start_time.weekday()

    return pd.Series({
        'user': group['user'].iloc[0],
        'trajectory_id': group['trajectory_id'].iloc[0],
        'start_time': start_time,
        'duration_s': duration,
        'start_hour': start_hour,
        'weekday': weekday,
        'distance_m': distance,
        'straightness_ratio': straightness,
        'avg_speed_mps': avg_speed,
        'speed_std': speed_std,
        'speed_entropy': speed_entropy,
        'speed_peak_ratio': speed_peak_ratio,
        'movement_ratio': movement_ratio,
        'load_factor': load_factor,
        'time_entropy': time_entropy,
        'vcr': speed_changes / norm_dist,
        'sr': stops / norm_dist,
        'hcr': heading_changes / norm_dist,
        'num_points': len(group),
        'coordinates': line,
        'centroid': line.centroid,
        'label': group['label'].mode().iloc[0] if 'label' in group.columns and not group['label'].mode().empty else None,
        'labels': group['label'].tolist() if 'label' in group.columns else None,
    })

aggregated_df = df.groupby(['user', 'trajectory_id']).apply(aggregate_activity).dropna().reset_index(drop=True)

/tmp/ipython-input-13-336625799.py:142: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  aggregated_df = df.groupby(['user', 'trajectory_id']).apply(aggregate_activity).dropna().reset_index(drop=True)


In [ ]:
end_times = df.groupby(['user', 'trajectory_id'])['time'].max().reset_index()
end_times = end_times.rename(columns={'time': 'end_time'})
aggregated_df_merged = aggregated_df.merge(end_times, on=['user', 'trajectory_id'], how='left')
aggregated_df_merged.count()

## Adding location information

For each trajectory, the centroid's coordinates are used to fetch the associated country and city.

In [ ]:
from geopy.geocoders import Nominatim
import time

trajectory_cities = {}
geolocator = Nominatim(user_agent="<user-agent>")

centroids = aggregated_df[['trajectory_id', 'centroid']].values
for trajectory_id, centroid in centroids:
  location = geolocator.reverse(centroid[7:-1].split(" "),  language="en")
  if location:
      country = location.raw['address']['country']
      city =  location.raw['address'].get('city') or location.raw['address'].get('town')
      loc = [country, city]
      print(f"Trajectory: {trajectory_id}. Location: {loc}")
      time.sleep(0.5)
      trajectory_cities[trajectory_id] = loc
  else:
      print(f"NOT FOUND: Trajectory: {trajectory_id}. Location: {location}")

In [ ]:
import json

with open("trajectory_cities.json", "w") as file:
  json.dump(trajectory_cities, file, indent=4)

In [ ]:
content = {}

with open("trajectory_cities.json") as json_data:
  d = json.load(json_data)
  for traj, loc in d.items():
    content[traj] = loc


fmt_content = []
for traj_id, loc in content.items():
  fmt_content.append({"trajectory_id": traj_id, "country": loc[0], "city": loc[1]})

locations = pd.DataFrame(fmt_content)
locations.to_csv("/content/drive/MyDrive/locations.csv")

In [24]:
all_df = pd.merge(locations, aggregated_df_merged, on='trajectory_id', how="right")
all_df.to_csv("/content/drive/MyDrive/all.csv", index=False)

# Subgroup Discovery

- TODO

In [ ]:
!pip install pysubgroup

In [36]:
to_use = [
    'country', 'city', 'duration_s',
    'weekday', 'distance_m', 'straightness_ratio',
    'avg_speed_mps', 'speed_std', 'speed_entropy', 'speed_peak_ratio',
    'movement_ratio', 'load_factor', 'time_entropy',
    'vcr', 'sr', 'hcr', 'label'
]
df_sd = all_df[to_use]

In [40]:
import pysubgroup as ps

In [74]:
# target = ps.NumericTarget(["straightness_ratio"])
target = ps.BinaryTarget('label', 6)
ignore = ['label']
search_space = ps.create_selectors(df_sd, ignore=ignore)

task = ps.SubgroupDiscoveryTask(
    data=df_sd,
    target=target,
    search_space=search_space,
    result_set_size=10,
    depth=3,
    qf=ps.WRAccQF(),#ps.StandardQFNumeric(a=1)
)

result = ps.DFS().execute(task)
result.to_dataframe()

,quality,subgroup,size_sg,size_dataset,positives_sg,positives_dataset,size_complement,relative_size_sg,relative_size_complement,coverage_sg,coverage_complement,target_share_sg,target_share_complement,target_share_dataset,lift
0,0.013887,avg_speed_mps>=7.49 AND country=='China' AND v...,355,4474,68,74,4119,0.079347,0.920653,0.918919,0.081081,0.191549,0.001457,0.01654,11.580967
1,0.013846,avg_speed_mps>=7.49 AND vcr<6.69,366,4474,68,74,4108,0.081806,0.918194,0.918919,0.081081,0.185792,0.001461,0.01654,11.232905
2,0.013467,avg_speed_mps>=7.49 AND speed_std>=6.60 AND vc...,287,4474,65,74,4187,0.064148,0.935852,0.878378,0.121622,0.226481,0.002150,0.01654,13.692909
3,0.013316,avg_speed_mps>=7.49 AND hcr<3.64 AND vcr<6.69,328,4474,65,74,4146,0.073312,0.926688,0.878378,0.121622,0.198171,0.002171,0.01654,11.981295
4,0.013116,country=='China' AND speed_std>=6.60 AND vcr<6.69,382,4474,65,74,4092,0.085382,0.914618,0.878378,0.121622,0.170157,0.002199,0.01654,10.287604
5,0.013079,speed_std>=6.60 AND vcr<6.69,392,4474,65,74,4082,0.087617,0.912383,0.878378,0.121622,0.165816,0.002205,0.01654,10.025165
6,0.012738,hcr<3.64 AND speed_std>=6.60 AND vcr<6.69,303,4474,62,74,4171,0.067725,0.932275,0.837838,0.162162,0.204620,0.002877,0.01654,12.371243
7,0.012678,avg_speed_mps>=7.49 AND country=='China' AND h...,561,4474,66,74,3913,0.125391,0.874609,0.891892,0.108108,0.117647,0.002044,0.01654,7.112878
8,0.012634,avg_speed_mps>=7.49 AND hcr<3.64,573,4474,66,74,3901,0.128073,0.871927,0.891892,0.108108,0.115183,0.002051,0.01654,6.963917
9,0.012604,country=='China' AND hcr<3.64 AND vcr<6.69,581,4474,66,74,3893,0.129861,0.870139,0.891892,0.108108,0.113597,0.002055,0.01654,6.868028
